In [1]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'Malgun Gothic'
pd.set_option('mode.chained_assignment',  None) # 경고를 끈다
pd.set_option('display.max_columns', None) # 모든 열 출력
pd.set_option('display.max_rows', None)
plt.rcParams['axes.unicode_minus'] = False

In [2]:
taas = pd.read_csv('data/taas/taas_modeling.csv', encoding='cp949')
taas_subset = pd.read_csv('data/taas/taas_final.csv', encoding='cp949')

In [3]:
taas.head(1)

,사고번호,가해운전자 차종,EPDO,이륜차,인구,기온,강수량,풍속,grid_id,단속카메라수,스쿨존수,음식점수,유흥주점수,단란주점수,교차로수,버스정류장수,사고일시_새벽,사고일시_오후,사고일시_저녁,사고일시_점심,사고일시_출근,사고일시_퇴근,요일_금요일,요일_목요일,요일_수요일,요일_월요일,요일_일요일,요일_토요일,요일_화요일,사고유형_차대사람,사고유형_차대차,사고유형_차량단독,법규위반_과속,법규위반_교차로운행방법위반,법규위반_기타,법규위반_보행자보호의무위반,법규위반_불법유턴,법규위반_신호위반,법규위반_안전거리미확보,법규위반_안전운전불이행,법규위반_중앙선침범,법규위반_직진우회전진행방해,법규위반_차로위반,노면상태_건조,노면상태_기타,노면상태_서리/결빙,노면상태_적설,노면상태_젖음/습기,노면상태_침수,노면상태_해빙,도로형태_교차로 - 교차로부근,도로형태_교차로 - 교차로안,도로형태_교차로 - 교차로횡단보도내,도로형태_기타 - 기타,도로형태_단일로 - 고가도로위,도로형태_단일로 - 교량,도로형태_단일로 - 기타,도로형태_단일로 - 지하차도(도로)내,도로형태_단일로 - 터널,도로형태_미분류 - 미분류,도로형태_주차장 - 주차장,가해운전자 연령_노년,가해운전자 연령_장년,가해운전자 연령_중년,가해운전자 연령_청년,가해운전자 연령_청소년
0,2019010100100002,이륜,3,15623.0,404049.0,-5.5,0.0,1.0,1980515.0,0.0,0.0,23.0,0.0,0.0,2.0,4.0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0


In [4]:
taas.shape

(61404, 66)

In [5]:
taas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61404 entries, 0 to 61403
Data columns (total 66 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   사고번호                  61404 non-null  int64  
 1   가해운전자 차종              61404 non-null  object 
 2   EPDO                  61404 non-null  int64  
 3   이륜차                   61404 non-null  float64
 4   인구                    61404 non-null  float64
 5   기온                    61404 non-null  float64
 6   강수량                   61404 non-null  float64
 7   풍속                    61404 non-null  float64
 8   grid_id               61404 non-null  float64
 9   단속카메라수                61404 non-null  float64
 10  스쿨존수                  61404 non-null  float64
 11  음식점수                  61404 non-null  float64
 12  유흥주점수                 61404 non-null  float64
 13  단란주점수                 61404 non-null  float64
 14  교차로수                  61404 non-null  float64
 15  버스정류장수             

### Scaling

In [6]:
taas.columns

Index(['사고번호', '가해운전자 차종', 'EPDO', '이륜차', '인구', '기온', '강수량', '풍속', 'grid_id',
       '단속카메라수', '스쿨존수', '음식점수', '유흥주점수', '단란주점수', '교차로수', '버스정류장수', '사고일시_새벽',
       '사고일시_오후', '사고일시_저녁', '사고일시_점심', '사고일시_출근', '사고일시_퇴근', '요일_금요일',
       '요일_목요일', '요일_수요일', '요일_월요일', '요일_일요일', '요일_토요일', '요일_화요일', '사고유형_차대사람',
       '사고유형_차대차', '사고유형_차량단독', '법규위반_과속', '법규위반_교차로운행방법위반', '법규위반_기타',
       '법규위반_보행자보호의무위반', '법규위반_불법유턴', '법규위반_신호위반', '법규위반_안전거리미확보',
       '법규위반_안전운전불이행', '법규위반_중앙선침범', '법규위반_직진우회전진행방해', '법규위반_차로위반', '노면상태_건조',
       '노면상태_기타', '노면상태_서리/결빙', '노면상태_적설', '노면상태_젖음/습기', '노면상태_침수', '노면상태_해빙',
       '도로형태_교차로 - 교차로부근', '도로형태_교차로 - 교차로안', '도로형태_교차로 - 교차로횡단보도내',
       '도로형태_기타 - 기타', '도로형태_단일로 - 고가도로위', '도로형태_단일로 - 교량', '도로형태_단일로 - 기타',
       '도로형태_단일로 - 지하차도(도로)내', '도로형태_단일로 - 터널', '도로형태_미분류 - 미분류',
       '도로형태_주차장 - 주차장', '가해운전자 연령_노년', '가해운전자 연령_장년', '가해운전자 연령_중년',
       '가해운전자 연령_청년', '가해운전자 연령_청소년'],
      dtype='object')

In [7]:
X = taas.drop('EPDO', axis=1)
y = taas['EPDO']

In [8]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
train_X.shape

(49123, 65)

In [10]:
test_X.shape

(12281, 65)

In [11]:
train_X.head()

,사고번호,가해운전자 차종,이륜차,인구,기온,강수량,풍속,grid_id,단속카메라수,스쿨존수,음식점수,유흥주점수,단란주점수,교차로수,버스정류장수,사고일시_새벽,사고일시_오후,사고일시_저녁,사고일시_점심,사고일시_출근,사고일시_퇴근,요일_금요일,요일_목요일,요일_수요일,요일_월요일,요일_일요일,요일_토요일,요일_화요일,사고유형_차대사람,사고유형_차대차,사고유형_차량단독,법규위반_과속,법규위반_교차로운행방법위반,법규위반_기타,법규위반_보행자보호의무위반,법규위반_불법유턴,법규위반_신호위반,법규위반_안전거리미확보,법규위반_안전운전불이행,법규위반_중앙선침범,법규위반_직진우회전진행방해,법규위반_차로위반,노면상태_건조,노면상태_기타,노면상태_서리/결빙,노면상태_적설,노면상태_젖음/습기,노면상태_침수,노면상태_해빙,도로형태_교차로 - 교차로부근,도로형태_교차로 - 교차로안,도로형태_교차로 - 교차로횡단보도내,도로형태_기타 - 기타,도로형태_단일로 - 고가도로위,도로형태_단일로 - 교량,도로형태_단일로 - 기타,도로형태_단일로 - 지하차도(도로)내,도로형태_단일로 - 터널,도로형태_미분류 - 미분류,도로형태_주차장 - 주차장,가해운전자 연령_노년,가해운전자 연령_장년,가해운전자 연령_중년,가해운전자 연령_청년,가해운전자 연령_청소년
57087,2020110500100025,이륜,14526.0,427684.0,4.3,0.0,0.7,1990459.0,2.0,0.0,11.0,0.0,0.0,6.0,0.0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
59182,2019082400100573,승용,25328.0,155511.0,23.2,0.0,0.7,4182038.0,0.0,0.0,35.0,5.0,7.0,4.0,0.0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
29830,2020073100100453,이륜,6487.0,175799.0,27.0,0.0,1.7,4064340.0,0.0,0.0,32.0,0.0,1.0,4.0,0.0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
44166,2019081600100444,승용,11559.0,424947.0,31.8,0.0,1.1,3550703.0,0.0,0.0,15.0,8.0,0.0,1.0,2.0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
27591,2019100900100357,화물,8660.0,272532.0,22.4,0.0,2.8,3981095.0,0.0,0.0,12.0,5.0,2.0,0.0,0.0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [12]:
test_X.head()

,사고번호,가해운전자 차종,이륜차,인구,기온,강수량,풍속,grid_id,단속카메라수,스쿨존수,음식점수,유흥주점수,단란주점수,교차로수,버스정류장수,사고일시_새벽,사고일시_오후,사고일시_저녁,사고일시_점심,사고일시_출근,사고일시_퇴근,요일_금요일,요일_목요일,요일_수요일,요일_월요일,요일_일요일,요일_토요일,요일_화요일,사고유형_차대사람,사고유형_차대차,사고유형_차량단독,법규위반_과속,법규위반_교차로운행방법위반,법규위반_기타,법규위반_보행자보호의무위반,법규위반_불법유턴,법규위반_신호위반,법규위반_안전거리미확보,법규위반_안전운전불이행,법규위반_중앙선침범,법규위반_직진우회전진행방해,법규위반_차로위반,노면상태_건조,노면상태_기타,노면상태_서리/결빙,노면상태_적설,노면상태_젖음/습기,노면상태_침수,노면상태_해빙,도로형태_교차로 - 교차로부근,도로형태_교차로 - 교차로안,도로형태_교차로 - 교차로횡단보도내,도로형태_기타 - 기타,도로형태_단일로 - 고가도로위,도로형태_단일로 - 교량,도로형태_단일로 - 기타,도로형태_단일로 - 지하차도(도로)내,도로형태_단일로 - 터널,도로형태_미분류 - 미분류,도로형태_주차장 - 주차장,가해운전자 연령_노년,가해운전자 연령_장년,가해운전자 연령_중년,가해운전자 연령_청년,가해운전자 연령_청소년
39690,2020082100100401,화물,23872.0,1317340.0,29.8,0.0,2.6,1934123.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
33777,2019012900100520,화물,6424.0,349868.0,3.4,0.0,0.9,2484006.0,0.0,0.0,119.0,0.0,0.0,4.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
280,2019011700100405,이륜,15956.0,406889.0,1.6,0.0,2.9,2143470.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
48457,2021010900100455,승용,11950.0,262451.0,-7.5,0.0,4.3,3425837.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
54116,2021050200100343,화물,12912.0,531031.0,15.3,0.0,3.8,1774132.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [13]:
train_X_new = train_X.drop(['이륜차', '인구', '기온', '강수량', '풍속', '단속카메라수', '스쿨존수', '음식점수', '유흥주점수', '단란주점수', '교차로수', '버스정류장수'], axis=1)
test_X_new = test_X.drop(['이륜차', '인구', '기온', '강수량', '풍속', '단속카메라수', '스쿨존수', '음식점수', '유흥주점수', '단란주점수', '교차로수', '버스정류장수'], axis=1)

In [14]:
train_X_new_cont = train_X[['이륜차', '인구', '기온', '강수량', '풍속', '단속카메라수', '스쿨존수', '음식점수', '유흥주점수', '단란주점수', '교차로수', '버스정류장수']]
test_X_new_cont = test_X[['이륜차', '인구', '기온', '강수량', '풍속', '단속카메라수', '스쿨존수', '음식점수', '유흥주점수', '단란주점수', '교차로수', '버스정류장수']]

In [15]:
train_X_new_cont.head()

,이륜차,인구,기온,강수량,풍속,단속카메라수,스쿨존수,음식점수,유흥주점수,단란주점수,교차로수,버스정류장수
57087,14526.0,427684.0,4.3,0.0,0.7,2.0,0.0,11.0,0.0,0.0,6.0,0.0
59182,25328.0,155511.0,23.2,0.0,0.7,0.0,0.0,35.0,5.0,7.0,4.0,0.0
29830,6487.0,175799.0,27.0,0.0,1.7,0.0,0.0,32.0,0.0,1.0,4.0,0.0
44166,11559.0,424947.0,31.8,0.0,1.1,0.0,0.0,15.0,8.0,0.0,1.0,2.0
27591,8660.0,272532.0,22.4,0.0,2.8,0.0,0.0,12.0,5.0,2.0,0.0,0.0


In [16]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()

train_X_new_cont_rbs = scaler.fit_transform(train_X_new_cont)
train_X_new_cont_rbs = pd.DataFrame(train_X_new_cont_rbs, columns=train_X_new_cont.columns)

test_X_new_cont_rbs = scaler.transform(test_X_new_cont)
test_X_new_cont_rbs = pd.DataFrame(test_X_new_cont_rbs, columns=test_X_new_cont.columns)

In [17]:
train_X_new_cont_rbs.head()

,이륜차,인구,기온,강수량,풍속,단속카메라수,스쿨존수,음식점수,유흥주점수,단란주점수,교차로수,버스정류장수
0,-0.034003,0.260688,-0.794872,0.0,-0.888889,2.0,0.0,0.190476,0.0,0.0,2.5,-0.333333
1,1.024498,-0.968689,0.416667,0.0,-0.888889,0.0,0.0,1.333333,5.0,7.0,1.5,-0.333333
2,-0.821754,-0.877050,0.660256,0.0,-0.333333,0.0,0.0,1.190476,0.0,1.0,1.5,-0.333333
3,-0.324743,0.248325,0.967949,0.0,-0.666667,0.0,0.0,0.380952,8.0,0.0,0.0,0.333333
4,-0.608819,-0.440117,0.365385,0.0,0.277778,0.0,0.0,0.238095,5.0,2.0,-0.5,-0.333333


In [18]:
test_X_new_cont_rbs.head()

,이륜차,인구,기온,강수량,풍속,단속카메라수,스쿨존수,음식점수,유흥주점수,단란주점수,교차로수,버스정류장수
0,0.881823,4.279171,0.839744,0.0,0.166667,0.0,0.0,-0.333333,0.0,0.0,-0.5,-0.333333
1,-0.827927,-0.090799,-0.852564,0.0,-0.777778,0.0,0.0,5.333333,0.0,0.0,1.5,-0.333333
2,0.106124,0.166759,-0.967949,0.0,0.333333,0.0,0.0,-0.333333,0.0,0.0,-0.5,0.333333
3,-0.286428,-0.485652,-1.551282,0.0,1.111111,0.0,0.0,-0.238095,0.0,0.0,0.0,-0.333333
4,-0.192161,0.727496,-0.089744,0.0,0.833333,0.0,0.0,-0.333333,0.0,0.0,-0.5,-0.333333


In [19]:
train_X_new_cont_rbs.reset_index(drop=True, inplace=True)
train_X_new.reset_index(drop=True, inplace=True)
test_X_new_cont_rbs.reset_index(drop=True, inplace=True)
test_X_new.reset_index(drop=True, inplace=True)
train_y.reset_index(drop=True, inplace=True)
test_y.reset_index(drop=True, inplace=True)

In [20]:
train_X = pd.concat([train_X_new_cont_rbs, train_X_new], axis=1)
test_X = pd.concat([test_X_new_cont_rbs, test_X_new], axis=1)

In [21]:
train_X_new_cont_rbs.shape, train_X_new.shape, train_X.shape

((49123, 12), (49123, 53), (49123, 65))

In [22]:
test_X_new_cont_rbs.shape, test_X_new.shape, test_X.shape

((12281, 12), (12281, 53), (12281, 65))

In [23]:
train_X.shape, test_X.shape, train_y.shape, test_y.shape

((49123, 65), (12281, 65), (49123,), (12281,))

In [24]:
train_X_1 = train_X.copy()
test_X_1 = test_X.copy()

In [25]:
train_X_1.head()

,이륜차,인구,기온,강수량,풍속,단속카메라수,스쿨존수,음식점수,유흥주점수,단란주점수,교차로수,버스정류장수,사고번호,가해운전자 차종,grid_id,사고일시_새벽,사고일시_오후,사고일시_저녁,사고일시_점심,사고일시_출근,사고일시_퇴근,요일_금요일,요일_목요일,요일_수요일,요일_월요일,요일_일요일,요일_토요일,요일_화요일,사고유형_차대사람,사고유형_차대차,사고유형_차량단독,법규위반_과속,법규위반_교차로운행방법위반,법규위반_기타,법규위반_보행자보호의무위반,법규위반_불법유턴,법규위반_신호위반,법규위반_안전거리미확보,법규위반_안전운전불이행,법규위반_중앙선침범,법규위반_직진우회전진행방해,법규위반_차로위반,노면상태_건조,노면상태_기타,노면상태_서리/결빙,노면상태_적설,노면상태_젖음/습기,노면상태_침수,노면상태_해빙,도로형태_교차로 - 교차로부근,도로형태_교차로 - 교차로안,도로형태_교차로 - 교차로횡단보도내,도로형태_기타 - 기타,도로형태_단일로 - 고가도로위,도로형태_단일로 - 교량,도로형태_단일로 - 기타,도로형태_단일로 - 지하차도(도로)내,도로형태_단일로 - 터널,도로형태_미분류 - 미분류,도로형태_주차장 - 주차장,가해운전자 연령_노년,가해운전자 연령_장년,가해운전자 연령_중년,가해운전자 연령_청년,가해운전자 연령_청소년
0,-0.034003,0.260688,-0.794872,0.0,-0.888889,2.0,0.0,0.190476,0.0,0.0,2.5,-0.333333,2020110500100025,이륜,1990459.0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,1.024498,-0.968689,0.416667,0.0,-0.888889,0.0,0.0,1.333333,5.0,7.0,1.5,-0.333333,2019082400100573,승용,4182038.0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,-0.821754,-0.877050,0.660256,0.0,-0.333333,0.0,0.0,1.190476,0.0,1.0,1.5,-0.333333,2020073100100453,이륜,4064340.0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,-0.324743,0.248325,0.967949,0.0,-0.666667,0.0,0.0,0.380952,8.0,0.0,0.0,0.333333,2019081600100444,승용,3550703.0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,-0.608819,-0.440117,0.365385,0.0,0.277778,0.0,0.0,0.238095,5.0,2.0,-0.5,-0.333333,2019100900100357,화물,3981095.0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [26]:
train_X = train_X.drop(['사고번호','가해운전자 차종','grid_id'], axis=1)
test_X = test_X.drop(['사고번호','가해운전자 차종','grid_id'], axis=1)

### 모델링

In [27]:
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_squared_log_error

### CatBoost

In [28]:
from catboost import CatBoostRegressor

cb = CatBoostRegressor(depth=7, learning_rate=0.01, random_state=42)
cb.fit(train_X, train_y)

print(f"RMSE for CatBoost : {round(mean_squared_error(test_y, cb.predict(test_X))**0.5,3)}")
print(f"RMSLE for CatBoost : {round(mean_squared_log_error(test_y, cb.predict(test_X))**0.5,3)}")

C:\Users\jwshi\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


0:	learn: 2.3204415	total: 166ms	remaining: 2m 45s
1:	learn: 2.3198740	total: 173ms	remaining: 1m 26s
2:	learn: 2.3193946	total: 179ms	remaining: 59.7s
3:	learn: 2.3188244	total: 186ms	remaining: 46.2s
4:	learn: 2.3183279	total: 192ms	remaining: 38.3s
5:	learn: 2.3178370	total: 198ms	remaining: 32.7s
6:	learn: 2.3173131	total: 204ms	remaining: 28.9s
7:	learn: 2.3167827	total: 210ms	remaining: 26s
8:	learn: 2.3162980	total: 215ms	remaining: 23.7s
9:	learn: 2.3158274	total: 221ms	remaining: 21.9s
10:	learn: 2.3153636	total: 227ms	remaining: 20.4s
11:	learn: 2.3148802	total: 233ms	remaining: 19.2s
12:	learn: 2.3144754	total: 238ms	remaining: 18.1s
13:	learn: 2.3141447	total: 242ms	remaining: 17s
14:	learn: 2.3136898	total: 248ms	remaining: 16.3s
15:	learn: 2.3132277	total: 253ms	remaining: 15.6s
16:	learn: 2.3127833	total: 259ms	remaining: 15s
17:	learn: 2.3123856	total: 265ms	remaining: 14.4s
18:	learn: 2.3119600	total: 271ms	remaining: 14s
19:	learn: 2.3115630	total: 276ms	remaining: 13

186:	learn: 2.2817324	total: 1.26s	remaining: 5.47s
187:	learn: 2.2816492	total: 1.26s	remaining: 5.47s
188:	learn: 2.2815697	total: 1.27s	remaining: 5.46s
189:	learn: 2.2814438	total: 1.28s	remaining: 5.45s
190:	learn: 2.2813352	total: 1.28s	remaining: 5.44s
191:	learn: 2.2812104	total: 1.29s	remaining: 5.43s
192:	learn: 2.2811116	total: 1.29s	remaining: 5.42s
193:	learn: 2.2810128	total: 1.3s	remaining: 5.41s
194:	learn: 2.2809200	total: 1.31s	remaining: 5.39s
195:	learn: 2.2808361	total: 1.31s	remaining: 5.38s
196:	learn: 2.2807627	total: 1.32s	remaining: 5.37s
197:	learn: 2.2806606	total: 1.32s	remaining: 5.36s
198:	learn: 2.2805811	total: 1.33s	remaining: 5.35s
199:	learn: 2.2804965	total: 1.33s	remaining: 5.34s
200:	learn: 2.2803957	total: 1.34s	remaining: 5.33s
201:	learn: 2.2803407	total: 1.34s	remaining: 5.31s
202:	learn: 2.2802699	total: 1.35s	remaining: 5.3s
203:	learn: 2.2801797	total: 1.35s	remaining: 5.29s
204:	learn: 2.2800876	total: 1.36s	remaining: 5.28s
205:	learn: 2.

346:	learn: 2.2701168	total: 2.16s	remaining: 4.07s
347:	learn: 2.2700659	total: 2.17s	remaining: 4.06s
348:	learn: 2.2700072	total: 2.17s	remaining: 4.05s
349:	learn: 2.2699423	total: 2.18s	remaining: 4.04s
350:	learn: 2.2699099	total: 2.18s	remaining: 4.04s
351:	learn: 2.2698536	total: 2.19s	remaining: 4.03s
352:	learn: 2.2697993	total: 2.19s	remaining: 4.02s
353:	learn: 2.2697540	total: 2.2s	remaining: 4.01s
354:	learn: 2.2697259	total: 2.21s	remaining: 4.01s
355:	learn: 2.2696815	total: 2.21s	remaining: 4s
356:	learn: 2.2696423	total: 2.22s	remaining: 3.99s
357:	learn: 2.2695889	total: 2.22s	remaining: 3.98s
358:	learn: 2.2694844	total: 2.23s	remaining: 3.98s
359:	learn: 2.2694465	total: 2.23s	remaining: 3.97s
360:	learn: 2.2693676	total: 2.24s	remaining: 3.96s
361:	learn: 2.2692993	total: 2.24s	remaining: 3.96s
362:	learn: 2.2692629	total: 2.25s	remaining: 3.95s
363:	learn: 2.2692148	total: 2.25s	remaining: 3.94s
364:	learn: 2.2691650	total: 2.26s	remaining: 3.93s
365:	learn: 2.26

506:	learn: 2.2616904	total: 3.06s	remaining: 2.98s
507:	learn: 2.2616732	total: 3.06s	remaining: 2.97s
508:	learn: 2.2616365	total: 3.07s	remaining: 2.96s
509:	learn: 2.2615943	total: 3.08s	remaining: 2.96s
510:	learn: 2.2615535	total: 3.08s	remaining: 2.95s
511:	learn: 2.2614957	total: 3.09s	remaining: 2.94s
512:	learn: 2.2614736	total: 3.09s	remaining: 2.94s
513:	learn: 2.2614167	total: 3.1s	remaining: 2.93s
514:	learn: 2.2613650	total: 3.1s	remaining: 2.92s
515:	learn: 2.2613009	total: 3.11s	remaining: 2.92s
516:	learn: 2.2612530	total: 3.12s	remaining: 2.91s
517:	learn: 2.2612086	total: 3.12s	remaining: 2.9s
518:	learn: 2.2611502	total: 3.13s	remaining: 2.9s
519:	learn: 2.2611024	total: 3.13s	remaining: 2.89s
520:	learn: 2.2610654	total: 3.14s	remaining: 2.88s
521:	learn: 2.2610176	total: 3.14s	remaining: 2.88s
522:	learn: 2.2609654	total: 3.15s	remaining: 2.87s
523:	learn: 2.2609090	total: 3.15s	remaining: 2.86s
524:	learn: 2.2608607	total: 3.16s	remaining: 2.86s
525:	learn: 2.26

668:	learn: 2.2547090	total: 3.96s	remaining: 1.96s
669:	learn: 2.2546503	total: 3.96s	remaining: 1.95s
670:	learn: 2.2546224	total: 3.97s	remaining: 1.95s
671:	learn: 2.2545884	total: 3.98s	remaining: 1.94s
672:	learn: 2.2545197	total: 3.98s	remaining: 1.93s
673:	learn: 2.2544765	total: 3.99s	remaining: 1.93s
674:	learn: 2.2543842	total: 3.99s	remaining: 1.92s
675:	learn: 2.2543699	total: 4s	remaining: 1.92s
676:	learn: 2.2543181	total: 4s	remaining: 1.91s
677:	learn: 2.2542885	total: 4.01s	remaining: 1.9s
678:	learn: 2.2542250	total: 4.01s	remaining: 1.9s
679:	learn: 2.2541915	total: 4.02s	remaining: 1.89s
680:	learn: 2.2541514	total: 4.02s	remaining: 1.89s
681:	learn: 2.2541073	total: 4.03s	remaining: 1.88s
682:	learn: 2.2540722	total: 4.04s	remaining: 1.87s
683:	learn: 2.2540107	total: 4.04s	remaining: 1.87s
684:	learn: 2.2539559	total: 4.05s	remaining: 1.86s
685:	learn: 2.2538973	total: 4.05s	remaining: 1.85s
686:	learn: 2.2538558	total: 4.06s	remaining: 1.85s
687:	learn: 2.253828

832:	learn: 2.2481447	total: 4.86s	remaining: 973ms
833:	learn: 2.2481040	total: 4.86s	remaining: 968ms
834:	learn: 2.2480468	total: 4.87s	remaining: 962ms
835:	learn: 2.2480290	total: 4.87s	remaining: 956ms
836:	learn: 2.2479800	total: 4.88s	remaining: 950ms
837:	learn: 2.2479550	total: 4.88s	remaining: 944ms
838:	learn: 2.2478961	total: 4.89s	remaining: 938ms
839:	learn: 2.2478256	total: 4.89s	remaining: 933ms
840:	learn: 2.2477812	total: 4.9s	remaining: 927ms
841:	learn: 2.2477408	total: 4.91s	remaining: 921ms
842:	learn: 2.2476666	total: 4.91s	remaining: 915ms
843:	learn: 2.2476139	total: 4.92s	remaining: 909ms
844:	learn: 2.2475554	total: 4.92s	remaining: 903ms
845:	learn: 2.2475161	total: 4.93s	remaining: 897ms
846:	learn: 2.2474366	total: 4.93s	remaining: 891ms
847:	learn: 2.2473826	total: 4.94s	remaining: 885ms
848:	learn: 2.2472930	total: 4.95s	remaining: 880ms
849:	learn: 2.2472642	total: 4.95s	remaining: 874ms
850:	learn: 2.2471977	total: 4.96s	remaining: 868ms
851:	learn: 2

994:	learn: 2.2418163	total: 5.75s	remaining: 28.9ms
995:	learn: 2.2417876	total: 5.75s	remaining: 23.1ms
996:	learn: 2.2417134	total: 5.76s	remaining: 17.3ms
997:	learn: 2.2416665	total: 5.76s	remaining: 11.6ms
998:	learn: 2.2416324	total: 5.77s	remaining: 5.78ms
999:	learn: 2.2415496	total: 5.78s	remaining: 0us
RMSE for CatBoost : 2.257
RMSLE for CatBoost : 0.397


### 가해운전자=이륜차 & 대전 지역 데이터프레임 생성

In [29]:
taas_subset = taas_subset[taas_subset['위치'].str.contains("대전광역시")]

In [30]:
daejeon_id = taas_subset['grid_id'].unique().tolist()

In [31]:
X_motor = pd.concat([train_X_1, test_X_1], axis=0)
y_motor = pd.concat([train_y, test_y], axis=0)

df = pd.concat([X_motor, y_motor], axis=1)

In [32]:
df.head()

,이륜차,인구,기온,강수량,풍속,단속카메라수,스쿨존수,음식점수,유흥주점수,단란주점수,교차로수,버스정류장수,사고번호,가해운전자 차종,grid_id,사고일시_새벽,사고일시_오후,사고일시_저녁,사고일시_점심,사고일시_출근,사고일시_퇴근,요일_금요일,요일_목요일,요일_수요일,요일_월요일,요일_일요일,요일_토요일,요일_화요일,사고유형_차대사람,사고유형_차대차,사고유형_차량단독,법규위반_과속,법규위반_교차로운행방법위반,법규위반_기타,법규위반_보행자보호의무위반,법규위반_불법유턴,법규위반_신호위반,법규위반_안전거리미확보,법규위반_안전운전불이행,법규위반_중앙선침범,법규위반_직진우회전진행방해,법규위반_차로위반,노면상태_건조,노면상태_기타,노면상태_서리/결빙,노면상태_적설,노면상태_젖음/습기,노면상태_침수,노면상태_해빙,도로형태_교차로 - 교차로부근,도로형태_교차로 - 교차로안,도로형태_교차로 - 교차로횡단보도내,도로형태_기타 - 기타,도로형태_단일로 - 고가도로위,도로형태_단일로 - 교량,도로형태_단일로 - 기타,도로형태_단일로 - 지하차도(도로)내,도로형태_단일로 - 터널,도로형태_미분류 - 미분류,도로형태_주차장 - 주차장,가해운전자 연령_노년,가해운전자 연령_장년,가해운전자 연령_중년,가해운전자 연령_청년,가해운전자 연령_청소년,EPDO
0,-0.034003,0.260688,-0.794872,0.0,-0.888889,2.0,0.0,0.190476,0.0,0.0,2.5,-0.333333,2020110500100025,이륜,1990459.0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,3
1,1.024498,-0.968689,0.416667,0.0,-0.888889,0.0,0.0,1.333333,5.0,7.0,1.5,-0.333333,2019082400100573,승용,4182038.0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,3
2,-0.821754,-0.877050,0.660256,0.0,-0.333333,0.0,0.0,1.190476,0.0,1.0,1.5,-0.333333,2020073100100453,이륜,4064340.0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,3
3,-0.324743,0.248325,0.967949,0.0,-0.666667,0.0,0.0,0.380952,8.0,0.0,0.0,0.333333,2019081600100444,승용,3550703.0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,3
4,-0.608819,-0.440117,0.365385,0.0,0.277778,0.0,0.0,0.238095,5.0,2.0,-0.5,-0.333333,2019100900100357,화물,3981095.0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,11


In [33]:
df = df[df['grid_id'].isin(daejeon_id)]

In [34]:
df.shape

(3467, 66)

In [35]:
df_1 = df[df['가해운전자 차종']=='이륜']
df_1.shape

(1426, 66)

In [36]:
df_1.head(1)

,이륜차,인구,기온,강수량,풍속,단속카메라수,스쿨존수,음식점수,유흥주점수,단란주점수,교차로수,버스정류장수,사고번호,가해운전자 차종,grid_id,사고일시_새벽,사고일시_오후,사고일시_저녁,사고일시_점심,사고일시_출근,사고일시_퇴근,요일_금요일,요일_목요일,요일_수요일,요일_월요일,요일_일요일,요일_토요일,요일_화요일,사고유형_차대사람,사고유형_차대차,사고유형_차량단독,법규위반_과속,법규위반_교차로운행방법위반,법규위반_기타,법규위반_보행자보호의무위반,법규위반_불법유턴,법규위반_신호위반,법규위반_안전거리미확보,법규위반_안전운전불이행,법규위반_중앙선침범,법규위반_직진우회전진행방해,법규위반_차로위반,노면상태_건조,노면상태_기타,노면상태_서리/결빙,노면상태_적설,노면상태_젖음/습기,노면상태_침수,노면상태_해빙,도로형태_교차로 - 교차로부근,도로형태_교차로 - 교차로안,도로형태_교차로 - 교차로횡단보도내,도로형태_기타 - 기타,도로형태_단일로 - 고가도로위,도로형태_단일로 - 교량,도로형태_단일로 - 기타,도로형태_단일로 - 지하차도(도로)내,도로형태_단일로 - 터널,도로형태_미분류 - 미분류,도로형태_주차장 - 주차장,가해운전자 연령_노년,가해운전자 연령_장년,가해운전자 연령_중년,가해운전자 연령_청년,가해운전자 연령_청소년,EPDO
43,-0.670848,-0.651955,-0.961538,0.0,-0.5,0.0,0.0,1.714286,0.0,0.0,0.0,-0.333333,2019122300100627,이륜,2452775.0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1


In [37]:
df_1 = df_1.sort_values(by='grid_id', ascending=True)

### grid_id별 사고 건수 column 생성

In [38]:
df_1['grid_id'].value_counts()

2430260.0    10
2480547.0    10
2492686.0     8
2399044.0     8
2494425.0     8
2490961.0     7
2393843.0     7
2477076.0     7
2397316.0     6
2440668.0     6
2404243.0     6
2399047.0     6
2409476.0     6
2480541.0     6
2400778.0     6
2430259.0     6
2480544.0     5
2442398.0     5
2482292.0     5
2392110.0     5
2400776.0     5
2430261.0     5
2399045.0     5
2466675.0     5
2431995.0     5
2470087.0     5
2397314.0     5
2475355.0     5
2419852.0     5
2395571.0     5
2489215.0     5
2397311.0     5
2473609.0     5
2468417.0     4
2397340.0     4
2499629.0     4
2487483.0     4
2431993.0     4
2418118.0     4
2494422.0     4
2397313.0     4
2487494.0     4
2425059.0     4
2435461.0     4
2490972.0     4
2402508.0     4
2435471.0     4
2442397.0     4
2423324.0     4
2423322.0     4
2423321.0     4
2497885.0     4
2485745.0     4
2405976.0     4
2442399.0     3
2449338.0     3
2397341.0     3
2399042.0     3
2447612.0     3
2447615.0     3
2482280.0     3
2428519.0     3
2482275.

In [39]:
value_counts_df = df_1['grid_id'].value_counts().reset_index()
value_counts_df.columns = ['grid_id', 'count']
value_counts_df = value_counts_df.sort_values(by='grid_id', ascending=True)
value_counts_df = value_counts_df.set_index('grid_id')

In [40]:
value_counts_df.head()

,count
grid_id,
2360893.0,1
2360897.0,1
2360900.0,1
2362631.0,1
2362636.0,1


In [41]:
df_2 = df_1.set_index('grid_id')

In [42]:
df_2.head()

,이륜차,인구,기온,강수량,풍속,단속카메라수,스쿨존수,음식점수,유흥주점수,단란주점수,교차로수,버스정류장수,사고번호,가해운전자 차종,사고일시_새벽,사고일시_오후,사고일시_저녁,사고일시_점심,사고일시_출근,사고일시_퇴근,요일_금요일,요일_목요일,요일_수요일,요일_월요일,요일_일요일,요일_토요일,요일_화요일,사고유형_차대사람,사고유형_차대차,사고유형_차량단독,법규위반_과속,법규위반_교차로운행방법위반,법규위반_기타,법규위반_보행자보호의무위반,법규위반_불법유턴,법규위반_신호위반,법규위반_안전거리미확보,법규위반_안전운전불이행,법규위반_중앙선침범,법규위반_직진우회전진행방해,법규위반_차로위반,노면상태_건조,노면상태_기타,노면상태_서리/결빙,노면상태_적설,노면상태_젖음/습기,노면상태_침수,노면상태_해빙,도로형태_교차로 - 교차로부근,도로형태_교차로 - 교차로안,도로형태_교차로 - 교차로횡단보도내,도로형태_기타 - 기타,도로형태_단일로 - 고가도로위,도로형태_단일로 - 교량,도로형태_단일로 - 기타,도로형태_단일로 - 지하차도(도로)내,도로형태_단일로 - 터널,도로형태_미분류 - 미분류,도로형태_주차장 - 주차장,가해운전자 연령_노년,가해운전자 연령_장년,가해운전자 연령_중년,가해운전자 연령_청년,가해운전자 연령_청소년,EPDO
grid_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2360893.0,-0.746399,-0.577322,0.698718,0.0,-0.222222,0.0,0.0,-0.285714,0.0,0.0,0.0,0.000000,2021062000100414,이륜,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,15
2360897.0,-0.766291,-0.088920,0.544872,0.0,-0.277778,0.0,0.0,-0.333333,0.0,0.0,0.0,-0.333333,2019091700100285,이륜,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1
2360900.0,-0.814993,-0.094783,-0.730769,0.0,-0.055556,0.0,0.0,-0.333333,0.0,0.0,1.5,-0.333333,2019032200100545,이륜,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,6
2362631.0,-0.846840,-0.871485,-0.179487,0.0,-0.888889,0.0,0.0,-0.333333,0.0,0.0,0.0,0.333333,2021110300100498,이륜,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,8
2362636.0,-0.517589,0.502297,0.455128,0.0,-1.000000,0.0,0.0,-0.285714,0.0,0.0,0.5,0.333333,2020091500100532,이륜,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,17


In [43]:
df_3 = df_2.drop(['사고번호','가해운전자 차종'], axis=1)

In [44]:
df_mean = df_3.groupby(df_3.index).mean()

In [45]:
df_mean.shape

(854, 63)

### grid_id별 최대 EPDO값 column 생성

In [46]:
df_max = df_3.groupby(df_3.index).max()

In [47]:
df_max = df_3.groupby(df_3.index).agg({'EPDO': 'max'}).reset_index()

In [48]:
df_max.rename(columns={'EPDO': 'max_EPDO'}, inplace=True)

In [49]:
df_max = df_max.set_index('grid_id')

In [50]:
df_max.head()

,max_EPDO
grid_id,
2360893.0,15
2360897.0,1
2360900.0,6
2362631.0,8
2362636.0,17


In [51]:
df_max.shape

(854, 1)

In [52]:
df_new = pd.merge(df_mean, df_max, on=df_mean.index, how='inner')

In [53]:
df_new.rename(columns={'key_0': 'grid_id'}, inplace=True)

In [54]:
df_new = df_new.set_index('grid_id')

In [55]:
df_new.head(5)

,이륜차,인구,기온,강수량,풍속,단속카메라수,스쿨존수,음식점수,유흥주점수,단란주점수,교차로수,버스정류장수,사고일시_새벽,사고일시_오후,사고일시_저녁,사고일시_점심,사고일시_출근,사고일시_퇴근,요일_금요일,요일_목요일,요일_수요일,요일_월요일,요일_일요일,요일_토요일,요일_화요일,사고유형_차대사람,사고유형_차대차,사고유형_차량단독,법규위반_과속,법규위반_교차로운행방법위반,법규위반_기타,법규위반_보행자보호의무위반,법규위반_불법유턴,법규위반_신호위반,법규위반_안전거리미확보,법규위반_안전운전불이행,법규위반_중앙선침범,법규위반_직진우회전진행방해,법규위반_차로위반,노면상태_건조,노면상태_기타,노면상태_서리/결빙,노면상태_적설,노면상태_젖음/습기,노면상태_침수,노면상태_해빙,도로형태_교차로 - 교차로부근,도로형태_교차로 - 교차로안,도로형태_교차로 - 교차로횡단보도내,도로형태_기타 - 기타,도로형태_단일로 - 고가도로위,도로형태_단일로 - 교량,도로형태_단일로 - 기타,도로형태_단일로 - 지하차도(도로)내,도로형태_단일로 - 터널,도로형태_미분류 - 미분류,도로형태_주차장 - 주차장,가해운전자 연령_노년,가해운전자 연령_장년,가해운전자 연령_중년,가해운전자 연령_청년,가해운전자 연령_청소년,EPDO,max_EPDO
grid_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2360893.0,-0.746399,-0.577322,0.698718,0.0,-0.222222,0.0,0.0,-0.285714,0.0,0.0,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,15.0,15
2360897.0,-0.766291,-0.088920,0.544872,0.0,-0.277778,0.0,0.0,-0.333333,0.0,0.0,0.0,-0.333333,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1
2360900.0,-0.814993,-0.094783,-0.730769,0.0,-0.055556,0.0,0.0,-0.333333,0.0,0.0,1.5,-0.333333,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,6.0,6
2362631.0,-0.846840,-0.871485,-0.179487,0.0,-0.888889,0.0,0.0,-0.333333,0.0,0.0,0.0,0.333333,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,8.0,8
2362636.0,-0.517589,0.502297,0.455128,0.0,-1.000000,0.0,0.0,-0.285714,0.0,0.0,0.5,0.333333,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,17.0,17


In [56]:
df_new_X = df_new.drop(['EPDO','max_EPDO'], axis=1) 

In [57]:
df_new_X.head()

,이륜차,인구,기온,강수량,풍속,단속카메라수,스쿨존수,음식점수,유흥주점수,단란주점수,교차로수,버스정류장수,사고일시_새벽,사고일시_오후,사고일시_저녁,사고일시_점심,사고일시_출근,사고일시_퇴근,요일_금요일,요일_목요일,요일_수요일,요일_월요일,요일_일요일,요일_토요일,요일_화요일,사고유형_차대사람,사고유형_차대차,사고유형_차량단독,법규위반_과속,법규위반_교차로운행방법위반,법규위반_기타,법규위반_보행자보호의무위반,법규위반_불법유턴,법규위반_신호위반,법규위반_안전거리미확보,법규위반_안전운전불이행,법규위반_중앙선침범,법규위반_직진우회전진행방해,법규위반_차로위반,노면상태_건조,노면상태_기타,노면상태_서리/결빙,노면상태_적설,노면상태_젖음/습기,노면상태_침수,노면상태_해빙,도로형태_교차로 - 교차로부근,도로형태_교차로 - 교차로안,도로형태_교차로 - 교차로횡단보도내,도로형태_기타 - 기타,도로형태_단일로 - 고가도로위,도로형태_단일로 - 교량,도로형태_단일로 - 기타,도로형태_단일로 - 지하차도(도로)내,도로형태_단일로 - 터널,도로형태_미분류 - 미분류,도로형태_주차장 - 주차장,가해운전자 연령_노년,가해운전자 연령_장년,가해운전자 연령_중년,가해운전자 연령_청년,가해운전자 연령_청소년
grid_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2360893.0,-0.746399,-0.577322,0.698718,0.0,-0.222222,0.0,0.0,-0.285714,0.0,0.0,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2360897.0,-0.766291,-0.088920,0.544872,0.0,-0.277778,0.0,0.0,-0.333333,0.0,0.0,0.0,-0.333333,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2360900.0,-0.814993,-0.094783,-0.730769,0.0,-0.055556,0.0,0.0,-0.333333,0.0,0.0,1.5,-0.333333,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2362631.0,-0.846840,-0.871485,-0.179487,0.0,-0.888889,0.0,0.0,-0.333333,0.0,0.0,0.0,0.333333,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2362636.0,-0.517589,0.502297,0.455128,0.0,-1.000000,0.0,0.0,-0.285714,0.0,0.0,0.5,0.333333,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [58]:
df_new_X[df_new_X['사고일시_새벽']==0.5]

,이륜차,인구,기온,강수량,풍속,단속카메라수,스쿨존수,음식점수,유흥주점수,단란주점수,교차로수,버스정류장수,사고일시_새벽,사고일시_오후,사고일시_저녁,사고일시_점심,사고일시_출근,사고일시_퇴근,요일_금요일,요일_목요일,요일_수요일,요일_월요일,요일_일요일,요일_토요일,요일_화요일,사고유형_차대사람,사고유형_차대차,사고유형_차량단독,법규위반_과속,법규위반_교차로운행방법위반,법규위반_기타,법규위반_보행자보호의무위반,법규위반_불법유턴,법규위반_신호위반,법규위반_안전거리미확보,법규위반_안전운전불이행,법규위반_중앙선침범,법규위반_직진우회전진행방해,법규위반_차로위반,노면상태_건조,노면상태_기타,노면상태_서리/결빙,노면상태_적설,노면상태_젖음/습기,노면상태_침수,노면상태_해빙,도로형태_교차로 - 교차로부근,도로형태_교차로 - 교차로안,도로형태_교차로 - 교차로횡단보도내,도로형태_기타 - 기타,도로형태_단일로 - 고가도로위,도로형태_단일로 - 교량,도로형태_단일로 - 기타,도로형태_단일로 - 지하차도(도로)내,도로형태_단일로 - 터널,도로형태_미분류 - 미분류,도로형태_주차장 - 주차장,가해운전자 연령_노년,가해운전자 연령_장년,가해운전자 연령_중년,가해운전자 연령_청년,가해운전자 연령_청소년
grid_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2376534.0,-0.623812,0.191993,-0.009615,0.000000,-0.916667,0.0,0.0,4.047619,0.0,0.0,0.5,1.000000,0.5,0.0,0.000000,0.000,0.50,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.500000,0.500,0.500000,0.500000,0.000,0.0,0.000,0.5,0.000,0.000,0.000,0.000,0.000000,0.500000,0.00,0.0,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.000,0.000,0.0,0.0,0.0,1.00,0.0,0.0,0.0,0.0,0.00,0.5,0.500000,0.000000,0.00
2397303.0,-0.655757,-0.191347,-1.310897,0.000000,-0.666667,0.0,0.0,-0.333333,0.0,0.0,-0.5,0.000000,0.5,0.0,0.000000,0.000,0.00,0.500000,0.000000,0.000000,0.00,0.500000,0.000000,0.500000,0.000,0.500000,0.500000,0.000,0.0,0.000,0.0,0.000,0.000,0.500,0.000,0.500000,0.000000,0.00,0.0,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.000,1.000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.500000,0.500000,0.00
2399044.0,-0.673236,-0.204151,0.300481,0.075000,-0.493056,0.0,0.0,0.904762,9.0,1.0,0.5,1.666667,0.5,0.0,0.250000,0.125,0.00,0.125000,0.250000,0.125000,0.25,0.000000,0.250000,0.000000,0.125,0.125000,0.750000,0.125,0.0,0.125,0.0,0.125,0.125,0.125,0.125,0.375000,0.000000,0.00,0.0,0.750000,0.0,0.0,0.0,0.250000,0.0,0.0,0.25,0.375,0.125,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.00,0.0,0.375000,0.625000,0.00
2405977.0,-0.804067,-0.086144,-0.019231,0.000000,-0.750000,1.0,0.0,0.904762,0.0,0.0,-0.5,1.000000,0.5,0.5,0.000000,0.000,0.00,0.000000,0.500000,0.500000,0.00,0.000000,0.000000,0.000000,0.000,0.500000,0.500000,0.000,0.0,0.000,0.0,0.000,0.000,0.000,0.000,1.000000,0.000000,0.00,0.0,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.000,0.000,0.0,0.0,0.0,1.00,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,1.000000,0.00
2407717.0,-0.702156,-0.615761,-0.179487,0.000000,-0.805556,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.5,0.0,0.000000,0.000,0.50,0.000000,0.000000,0.000000,0.50,0.000000,0.000000,0.000000,0.500,0.000000,1.000000,0.000,0.0,0.000,0.0,0.000,0.000,0.000,0.000,1.000000,0.000000,0.00,0.0,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.50,0.500,0.000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,1.000000,0.00
2412914.0,-0.763057,-0.335104,0.644231,0.000000,-1.000000,0.0,0.0,-0.333333,0.0,0.0,1.0,-0.333333,0.5,0.0,0.000000,0.000,0.00,0.500000,0.500000,0.000000,0.00,0.000000,0.000000,0.000000,0.500,0.000000,1.000000,0.000,0.0,0.000,0.0,0.000,0.000,0.000,0.000,1.000000,0.000000,0.00,0.0,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.000,0.000,0.0,0.0,0.0,1.00,0.0,0.0,0.0,0.0,0.00,0.0,0.500000,0.500000,0.00
2416382.0,-0.634150,-0.041059,0.733974,0.000000,-0.416667,0.0,0.0,-0.190476,0.0,0.0,0.0,0.333333,0.5,0.5,0.000000,0.000,0.00,0.000000,0.500000,0.000000,0.00,0.000000,0.500000,0.000000,0.000,0.000000,0.500000,0.500,0.0,0.000,0.0,0.000,0.000,0.500,0.000,0.500000,0.000000,0.00,0.0,0.500000,0.0,0.0,0.0,0.500000,0.0,0.0,0.00,0.500,0.000,0.0,0.0,0.0,0.50,0.0,0.0,0.0,0.0,0.00,0.0,0.500000,0.500000,0.00
2423322.0,-0.614356,-0.357301,-0.304487,0.000000,-0.083333,0.0,0.0,0.190476,0.0,0.0,-0.5,-0.333333,0.5,0.0,0.500000,0.000,0.00,0.000000,0.000000,0.250000,0.50,0.000000,0.000000,0.250000,0.000,0.000000,1.000000,0.000,0.0,0.000,0.0,0.000,0.000,0.000,0.250,0.250000,0.250000,0.25,0.0,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.25,0.250,0.000,0.0,0.0,0.0,0.50,0.0,0.0,0.0,0.0,0.00,0.0,0.500000,0.250000,0.25
2428539.0,-0.694904,-0.638262,-0.621795,0.000000,-0.305556,0.0,2.0,0.190476,0.0,0.0,-0.5,0.333333,0.5,0.0,0.500000,0.000,0

### 평균값으로 대체된 더미 변수들 전처리
+ 0.5이하일 경우 : 0
+ 0.5 초과일 경우 : 1

In [59]:
import numpy as np

df_new_X.iloc[:, 12:] = np.where(df_new_X.iloc[:, 12:] <= 0.5, 0, 1)

In [60]:
df_new_X.shape

(854, 62)

In [61]:
df_new_X.describe()

,이륜차,인구,기온,강수량,풍속,단속카메라수,스쿨존수,음식점수,유흥주점수,단란주점수,교차로수,버스정류장수,사고일시_새벽,사고일시_오후,사고일시_저녁,사고일시_점심,사고일시_출근,사고일시_퇴근,요일_금요일,요일_목요일,요일_수요일,요일_월요일,요일_일요일,요일_토요일,요일_화요일,사고유형_차대사람,사고유형_차대차,사고유형_차량단독,법규위반_과속,법규위반_교차로운행방법위반,법규위반_기타,법규위반_보행자보호의무위반,법규위반_불법유턴,법규위반_신호위반,법규위반_안전거리미확보,법규위반_안전운전불이행,법규위반_중앙선침범,법규위반_직진우회전진행방해,법규위반_차로위반,노면상태_건조,노면상태_기타,노면상태_서리/결빙,노면상태_적설,노면상태_젖음/습기,노면상태_침수,노면상태_해빙,도로형태_교차로 - 교차로부근,도로형태_교차로 - 교차로안,도로형태_교차로 - 교차로횡단보도내,도로형태_기타 - 기타,도로형태_단일로 - 고가도로위,도로형태_단일로 - 교량,도로형태_단일로 - 기타,도로형태_단일로 - 지하차도(도로)내,도로형태_단일로 - 터널,도로형태_미분류 - 미분류,도로형태_주차장 - 주차장,가해운전자 연령_노년,가해운전자 연령_장년,가해운전자 연령_중년,가해운전자 연령_청년,가해운전자 연령_청소년
count,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.0,854.0,854.000000,854.000000,854.000000,854.000000,854.000000,854.000000,854.00000,854.000000,854.0,854.0,854.0,854.000000,854.000000,854.000000,854.000000,854.000000
mean,-0.653908,-0.116056,-0.035283,0.151904,-0.390627,0.243560,0.228337,0.169120,0.241218,0.222482,-0.018735,0.018735,0.069087,0.098361,0.207260,0.133489,0.080796,0.140515,0.129977,0.099532,0.099532,0.084309,0.094848,0.106557,0.096019,0.126464,0.754098,0.036300,0.002342,0.042155,0.033958,0.017564,0.002342,0.159251,0.045667,0.457845,0.021077,0.016393,0.003513,0.884075,0.002342,0.002342,0.001171,0.067916,0.0,0.0,0.111241,0.312646,0.031616,0.025761,0.004684,0.002342,0.32904,0.001171,0.0,0.0,0.0,0.071429,0.118267,0.288056,0.270492,0.033958
std,0.119478,0.465934,0.556912,0.965496,0.530272,0.525258,0.664344,0.831084,1.404360,0.987453,0.589275,0.482237,0.253750,0.297976,0.405581,0.340302,0.272682,0.347724,0.336475,0.299550,0.299550,0.278014,0.293176,0.308730,0.294790,0.332566,0.430873,0.187145,0.048365,0.201059,0.181227,0.131439,0.048365,0.366124,0.208885,0.498512,0.143726,0.127057,0.059200,0.320323,0.048365,0.048365,0.034219,0.251749,0.0,0.0,0.314615,0.463843,0.175078,0.158515,0.068318,0.048365,0.47014,0.034219,0.0,0.0,0.0,0.257690,0.323113,0.453123,0.444474,0.181227
min,-0.858403,-0.884105,-1.769231,0.000000,-1.277778,0.000000,0.000000,-0.333333,0.000000,0.000000,-0.500000,-0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-0.749633,-0.597034,-0.435897,0.000000,-0.777778,0.000000,0.000000,-0.333333,0.000000,0.000000,-0.500000,-0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
50%,-0.663498,-0.089995,0.002404,0.000000,-0.500000,0.000000,0.000000,-0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
75%,-0.531308,0.471898

In [62]:
df_new_X.head()

,이륜차,인구,기온,강수량,풍속,단속카메라수,스쿨존수,음식점수,유흥주점수,단란주점수,교차로수,버스정류장수,사고일시_새벽,사고일시_오후,사고일시_저녁,사고일시_점심,사고일시_출근,사고일시_퇴근,요일_금요일,요일_목요일,요일_수요일,요일_월요일,요일_일요일,요일_토요일,요일_화요일,사고유형_차대사람,사고유형_차대차,사고유형_차량단독,법규위반_과속,법규위반_교차로운행방법위반,법규위반_기타,법규위반_보행자보호의무위반,법규위반_불법유턴,법규위반_신호위반,법규위반_안전거리미확보,법규위반_안전운전불이행,법규위반_중앙선침범,법규위반_직진우회전진행방해,법규위반_차로위반,노면상태_건조,노면상태_기타,노면상태_서리/결빙,노면상태_적설,노면상태_젖음/습기,노면상태_침수,노면상태_해빙,도로형태_교차로 - 교차로부근,도로형태_교차로 - 교차로안,도로형태_교차로 - 교차로횡단보도내,도로형태_기타 - 기타,도로형태_단일로 - 고가도로위,도로형태_단일로 - 교량,도로형태_단일로 - 기타,도로형태_단일로 - 지하차도(도로)내,도로형태_단일로 - 터널,도로형태_미분류 - 미분류,도로형태_주차장 - 주차장,가해운전자 연령_노년,가해운전자 연령_장년,가해운전자 연령_중년,가해운전자 연령_청년,가해운전자 연령_청소년
grid_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2360893.0,-0.746399,-0.577322,0.698718,0.0,-0.222222,0.0,0.0,-0.285714,0.0,0.0,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2360897.0,-0.766291,-0.088920,0.544872,0.0,-0.277778,0.0,0.0,-0.333333,0.0,0.0,0.0,-0.333333,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2360900.0,-0.814993,-0.094783,-0.730769,0.0,-0.055556,0.0,0.0,-0.333333,0.0,0.0,1.5,-0.333333,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2362631.0,-0.846840,-0.871485,-0.179487,0.0,-0.888889,0.0,0.0,-0.333333,0.0,0.0,0.0,0.333333,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2362636.0,-0.517589,0.502297,0.455128,0.0,-1.000000,0.0,0.0,-0.285714,0.0,0.0,0.5,0.333333,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


### 그리드별 EPDO 예측

In [63]:
result = cb.predict(df_new_X)
result = pd.DataFrame(result, columns=['EPDO_reg'])

In [64]:
result.head()

,EPDO_reg
0,4.259322
1,4.370910
2,4.078577
3,4.187592
4,4.748920


In [65]:
result.index = df_new_X.index

In [66]:
df_final = pd.concat([df_new, result], axis=1)

In [67]:
df_final = pd.concat([df_final, value_counts_df], axis=1)

In [68]:
df_final.head()

,이륜차,인구,기온,강수량,풍속,단속카메라수,스쿨존수,음식점수,유흥주점수,단란주점수,교차로수,버스정류장수,사고일시_새벽,사고일시_오후,사고일시_저녁,사고일시_점심,사고일시_출근,사고일시_퇴근,요일_금요일,요일_목요일,요일_수요일,요일_월요일,요일_일요일,요일_토요일,요일_화요일,사고유형_차대사람,사고유형_차대차,사고유형_차량단독,법규위반_과속,법규위반_교차로운행방법위반,법규위반_기타,법규위반_보행자보호의무위반,법규위반_불법유턴,법규위반_신호위반,법규위반_안전거리미확보,법규위반_안전운전불이행,법규위반_중앙선침범,법규위반_직진우회전진행방해,법규위반_차로위반,노면상태_건조,노면상태_기타,노면상태_서리/결빙,노면상태_적설,노면상태_젖음/습기,노면상태_침수,노면상태_해빙,도로형태_교차로 - 교차로부근,도로형태_교차로 - 교차로안,도로형태_교차로 - 교차로횡단보도내,도로형태_기타 - 기타,도로형태_단일로 - 고가도로위,도로형태_단일로 - 교량,도로형태_단일로 - 기타,도로형태_단일로 - 지하차도(도로)내,도로형태_단일로 - 터널,도로형태_미분류 - 미분류,도로형태_주차장 - 주차장,가해운전자 연령_노년,가해운전자 연령_장년,가해운전자 연령_중년,가해운전자 연령_청년,가해운전자 연령_청소년,EPDO,max_EPDO,EPDO_reg,count
grid_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2360893.0,-0.746399,-0.577322,0.698718,0.0,-0.222222,0.0,0.0,-0.285714,0.0,0.0,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,15.0,15,4.259322,1
2360897.0,-0.766291,-0.088920,0.544872,0.0,-0.277778,0.0,0.0,-0.333333,0.0,0.0,0.0,-0.333333,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1,4.370910,1
2360900.0,-0.814993,-0.094783,-0.730769,0.0,-0.055556,0.0,0.0,-0.333333,0.0,0.0,1.5,-0.333333,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,6.0,6,4.078577,1
2362631.0,-0.846840,-0.871485,-0.179487,0.0,-0.888889,0.0,0.0,-0.333333,0.0,0.0,0.0,0.333333,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,8.0,8,4.187592,1
2362636.0,-0.517589,0.502297,0.455128,0.0,-1.000000,0.0,0.0,-0.285714,0.0,0.0,0.5,0.333333,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,17.0,17,4.748920,1


In [69]:
df_final_sub = df_final[['EPDO','max_EPDO','EPDO_reg','count']]

In [70]:
df_final_sub.head()

,EPDO,max_EPDO,EPDO_reg,count
grid_id,,,,
2360893.0,15.0,15,4.259322,1
2360897.0,1.0,1,4.370910,1
2360900.0,6.0,6,4.078577,1
2362631.0,8.0,8,4.187592,1
2362636.0,17.0,17,4.748920,1


In [71]:
mean_squared_error(df_final_sub['EPDO'], df_final_sub['EPDO_reg'])**0.5

2.5732024094921635

In [72]:
df_final_sub['EPDO_reg'].describe()

count    854.000000
mean       4.261184
std        0.343259
min        3.415044
25%        4.035330
50%        4.170606
75%        4.401763
max        6.790905
Name: EPDO_reg, dtype: float64

In [73]:
df_final_sub = df_final_sub[df_final_sub['count']>=2]

### 최종 위험도 산출

In [74]:
df_final_sub['danger'] = ((df_final_sub['EPDO_reg']-df_final_sub['EPDO'])*df_final_sub['EPDO']*df_final_sub['count'])/df_final_sub['max_EPDO']

In [75]:
df_final_sub = df_final_sub.sort_values(by='danger', ascending=False)

In [76]:
df_final_sub.head()

,EPDO,max_EPDO,EPDO_reg,count,danger
grid_id,,,,,
2442397.0,2.000000,3,4.222009,4,5.925356
2404243.0,3.000000,3,3.911659,6,5.469957
2397313.0,3.000000,3,4.318848,4,5.275394
2482275.0,3.000000,3,4.654945,3,4.964835
2452802.0,2.333333,3,4.364635,3,4.739703


In [77]:
df_final_sub.describe()

,EPDO,max_EPDO,EPDO_reg,count,danger
count,319.000000,319.000000,319.000000,319.000000,319.000000
mean,4.614741,6.451411,4.153211,2.793103,-0.800605
std,1.693472,3.109630,0.232635,1.320312,3.053326
min,2.000000,3.000000,3.606623,2.000000,-15.954534
25%,3.366667,5.000000,4.014312,2.000000,-2.289134
50%,4.500000,6.000000,4.114330,2.000000,-0.331422
75%,5.500000,8.000000,4.251385,3.000000,1.669682
max,12.000000,18.000000,4.997837,10.000000,5.925356


In [78]:
# 최종 위험도 내보내기
df_final_sub.to_csv('data/danger_grid.csv', encoding = 'cp949')

### 위험도 1위 그리드 : 2442397

In [79]:
import skimage
import shap

In [80]:
explainer = shap.TreeExplainer(cb)
shap_values = explainer.shap_values(df_new_X)

In [81]:
df_new_X.index.get_loc(2442397.0)

424

In [82]:
shap.initjs() # javascript 초기화 (graph 초기화)
shap.force_plot(explainer.expected_value, shap_values[424,:], df_new_X.iloc[424,:])

In [83]:
taas_subset_1 = taas_subset[taas_subset['가해운전자 차종']=='이륜']

In [84]:
taas_subset_1[taas_subset_1['grid_id']==2442397.0]

,사고번호,사고일시,요일,시군구,사고유형,법규위반,노면상태,기상상태,도로형태,가해운전자 차종,가해운전자 성별,가해운전자 연령,x,y,EPDO,연월,위치,이륜차,인구,기온,강수량,풍속,grid_id,단속카메라수,스쿨존수,음식점수,유흥주점수,단란주점수,교차로수,버스정류장수
34051,2019050400100393,201905041400,토요일,대전광역시 서구 둔산동,차대차,안전거리미확보,건조,맑음,교차로 - 교차로안,이륜,남,중년,127.390288,36.351341,1,201905,대전광역시 서구,6750,351882,26.7,0.0,1.4,2442397.0,0.0,1.0,0.0,0.0,0.0,6,2.0
35401,2020070700100575,202007072200,화요일,대전광역시 서구 둔산동,차대차,안전운전불이행,건조,맑음,교차로 - 교차로부근,이륜,남,청년,127.390177,36.351359,1,202007,대전광역시 서구,9451,481763,23.9,0.0,0.9,2442397.0,0.0,1.0,0.0,0.0,0.0,6,2.0
35414,2020071100100622,202007112200,토요일,대전광역시 서구 둔산동,차대차,신호위반,건조,맑음,교차로 - 교차로횡단보도내,이륜,남,청년,127.390422,36.351242,3,202007,대전광역시 서구,9494,481964,26.2,0.0,0.8,2442397.0,0.0,1.0,0.0,0.0,0.0,6,2.0
36437,2021052900100373,202105291600,토요일,대전광역시 서구 둔산동,차대차,안전운전불이행,건조,맑음,교차로 - 교차로부근,이륜,남,청소년,127.390467,36.351278,3,202105,대전광역시 서구,10306,473365,24.0,0.0,2.9,2442397.0,0.0,1.0,0.0,0.0,0.0,6,2.0


### 위험도 2위 그리드 : 2404243

In [85]:
df_new_X.index.get_loc(2404243.0)

160

In [86]:
shap.initjs() # javascript 초기화 (graph 초기화)
shap.force_plot(explainer.expected_value, shap_values[160,:], df_new_X.iloc[160,:])

In [87]:
taas_subset_1[taas_subset_1['grid_id']==2404243.0]

,사고번호,사고일시,요일,시군구,사고유형,법규위반,노면상태,기상상태,도로형태,가해운전자 차종,가해운전자 성별,가해운전자 연령,x,y,EPDO,연월,위치,이륜차,인구,기온,강수량,풍속,grid_id,단속카메라수,스쿨존수,음식점수,유흥주점수,단란주점수,교차로수,버스정류장수
34737,2019120800100465,201912082000,일요일,대전광역시 유성구 궁동,차대사람,신호위반,건조,맑음,교차로 - 교차로횡단보도내,이륜,남,청년,127.347472,36.361463,3,201912,대전광역시 유성구,8044,227467,1.3,0.0,0.0,2404243.0,2.0,0.0,139.0,0.0,2.0,0,1.0
35855,2020112700100489,202011271800,금요일,대전광역시 유성구 궁동,차대차,안전운전불이행,건조,맑음,단일로 - 기타,이륜,남,청년,127.347327,36.361373,3,202011,대전광역시 유성구,9864,481124,5.5,0.0,2.7,2404243.0,2.0,0.0,139.0,0.0,2.0,0,1.0
36113,2021022000100124,202102201000,토요일,대전광역시 유성구 궁동,차대차,안전운전불이행,건조,맑음,단일로 - 기타,이륜,남,청년,127.346446,36.361786,3,202102,대전광역시 유성구,10047,475622,11.0,0.0,2.5,2404243.0,2.0,0.0,139.0,0.0,2.0,0,1.0
36184,2021031300100419,202103131800,토요일,대전광역시 유성구 궁동,차대차,중앙선침범,건조,맑음,단일로 - 기타,이륜,남,청년,127.346491,36.361471,3,202103,대전광역시 유성구,10112,474934,13.5,0.0,1.9,2404243.0,2.0,0.0,139.0,0.0,2.0,0,1.0
37026,2021111000100384,202111101600,수요일,대전광역시 유성구 궁동,차대차,안전거리미확보,젖음/습기,비,기타 - 기타,이륜,남,장년,127.346234,36.361957,3,202111,대전광역시 유성구,6241,175452,7.7,0.0,1.0,2404243.0,2.0,0.0,139.0,0.0,2.0,0,1.0
37168,2021122400100448,202112241800,금요일,대전광역시 유성구 궁동,차대차,교차로운행방법위반,건조,맑음,교차로 - 교차로안,이륜,남,청년,127.347472,36.361454,3,202112,대전광역시 유성구,6128,174662,3.0,0.0,4.1,2404243.0,2.0,0.0,139.0,0.0,2.0,0,1.0


### 위험도 3위 그리드 : 2397313

In [88]:
df_new_X.index.get_loc(2397313.0)

115

In [89]:
shap.initjs() # javascript 초기화 (graph 초기화)
shap.force_plot(explainer.expected_value, shap_values[115,:], df_new_X.iloc[115,:])

In [90]:
taas_subset_1[taas_subset_1['grid_id']==2397313.0]

,사고번호,사고일시,요일,시군구,사고유형,법규위반,노면상태,기상상태,도로형태,가해운전자 차종,가해운전자 성별,가해운전자 연령,x,y,EPDO,연월,위치,이륜차,인구,기온,강수량,풍속,grid_id,단속카메라수,스쿨존수,음식점수,유흥주점수,단란주점수,교차로수,버스정류장수
34631,2019110600100604,201911061800,수요일,대전광역시 유성구 봉명동,차대차,안전운전불이행,건조,맑음,단일로 - 기타,이륜,남,청년,127.340381,36.351465,3,201911,대전광역시 유성구,7924,229309,14.7,0.0,1.1,2397313.0,0.0,0.0,1.0,0.0,0.0,0,1.0
36013,2021012200100195,202101221300,금요일,대전광역시 유성구 봉명동,차대차,신호위반,건조,맑음,교차로 - 교차로안,이륜,남,중년,127.340116,36.350238,3,202101,대전광역시 유성구,9933,477326,9.7,0.0,0.7,2397313.0,0.0,0.0,1.0,0.0,0.0,0,1.0
36645,2021072700100231,202107271100,화요일,대전광역시 유성구 봉명동,차대차,안전운전불이행,건조,맑음,교차로 - 교차로안,이륜,남,장년,127.339611,36.351924,3,202107,대전광역시 유성구,7267,239361,33.0,0.0,3.4,2397313.0,0.0,0.0,1.0,0.0,0.0,0,1.0
36903,2021100600100172,202110061000,수요일,대전광역시 유성구 봉명동,차대차,신호위반,건조,맑음,교차로 - 교차로안,이륜,남,청년,127.340160,36.350527,3,202110,대전광역시 유성구,7468,231306,23.9,0.0,2.2,2397313.0,0.0,0.0,1.0,0.0,0.0,0,1.0


In [92]:
df_final_sub.iloc[302]

EPDO         7.333333
max_EPDO    17.000000
EPDO_reg     4.864913
count        6.000000
danger      -6.388853
Name: 2399047.0, dtype: float64